In [2]:
import os
import json
import numpy as np

models_dir = "/Users/beatriz.herrera/Documents/Python-Scripts/arbor-bmtk/bionet-VISp-biocells-syns/components/biophysical_neuron_templates/ctdb"
model_saving_dir = "components/biophysical_neuron_templates/ctdb"

In [3]:
modfiles = os.listdir("components/mechanisms/modfiles")
# ion_channels_pass = ["g_pas"]
ion_channels = [f.split(".")[0] for f in modfiles if f.endswith(".mod")]
# ion_channels = ion_channels_pass + ion_channels_mod

In [4]:
for model in os.listdir(models_dir):
    if model.endswith(".json"):
        for ion_channel in ion_channels:
            genome_name = (
                "gbar_{}".format(ion_channel) if ion_channel != "g_pas" else "g_pas"
            )

            with open(os.path.join(models_dir, model), "r") as f:
                modeli = json.load(f)

            model_mechanisms = [channel["name"] for channel in modeli["genome"]]

            if ~np.isin(genome_name, model_mechanisms):
                continue

            # modeli_updated = modeli.copy()

            # mechanims2update = [
            #     channel for channel in model_mechanisms if channel.startswith(ion_channel)
            # ]
            def genome_update(mechanism):
                if (
                    mechanism["name"].startswith("gbar_")
                    and mechanism["name"] != genome_name
                    and mechanism["name"] != "g_pas"
                ):
                    mechanism["value"] = 0.0
                return mechanism

            modeli["genome"] = [genome_update(channel) for channel in modeli["genome"]]

            with open(
                os.path.join(
                    model_saving_dir, model.split(".")[0] + "_" + ion_channel + ".json"
                ),
                "w",
            ) as f:
                json.dump(modeli, f, indent=2)

In [5]:
all_models = os.listdir(model_saving_dir)

with open(
    "model_props/v1_node_models.biophysical_simplified.json",
    "r",
) as f:
    v1_node_models = json.load(f)

node_type_id = 100000000
for layer_name, layer in v1_node_models["locations"].items():
    for pop_name, pop in layer.items():
        main_model_name = pop["models"][0]["dynamics_params"].split(".")[0]

        model_files = [m for m in all_models if m.startswith(main_model_name)]

        dict_models = []

        ref_dict = pop["models"][0].copy()
        for ii in range(len(model_files)):
            modeli_dict = ref_dict.copy()

            if model_files[ii].split(".")[0] != main_model_name:
                node_type_id += 1
                modeli_dict.update(
                    {
                        "dynamics_params": model_files[ii],
                        "node_type_id": node_type_id,
                    }
                )

            dict_models.append(modeli_dict)

        v1_node_models["locations"][layer_name][pop_name]["models"] = dict_models

In [6]:
v1_node_models["locations"]

{'VisL1': {'i1Htr3a': {'ncells': 4363,
   'ei': 'i',
   'depth_range': [50.0, 100.0],
   'models': [{'node_type_id': 100000001,
     'model_type': 'biophysical',
     'model_template': 'ctdb:Biophys1.hoc',
     'dynamics_params': '478107198_fit_Ca_LVA.json',
     'morphology': 'Htr3a-Cre_NO152_Ai14-192819.04.02.01_491119743_m.swc',
     'model_processing': 'aibs_perisomatic',
     'rotation_angle_zaxis': -3.465523998},
    {'node_type_id': 100000002,
     'model_type': 'biophysical',
     'model_template': 'ctdb:Biophys1.hoc',
     'dynamics_params': '478107198_fit_Kv2like.json',
     'morphology': 'Htr3a-Cre_NO152_Ai14-192819.04.02.01_491119743_m.swc',
     'model_processing': 'aibs_perisomatic',
     'rotation_angle_zaxis': -3.465523998},
    {'node_type_id': 100000003,
     'model_type': 'biophysical',
     'model_template': 'ctdb:Biophys1.hoc',
     'dynamics_params': '478107198_fit_Kd.json',
     'morphology': 'Htr3a-Cre_NO152_Ai14-192819.04.02.01_491119743_m.swc',
     'model_pro

In [7]:
with open(
    "model_props/v1_node_models.biophysical_simplified.mechanism_check.json",
    "w",
) as f:
    json.dump(v1_node_models, f, indent=2)